Test by the following command if your **GPU** is working.

In [22]:
!nvidia-smi

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



Mount Your **Google Drive** by executing the following code.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [9]:
cd "My Drive"


[Errno 2] No such file or directory: 'My Drive'
/content/drive/My Drive


Clone the Source Code from **Github** and Install **Requirements** using **PIP**

In [11]:
!git clone https://github.com/Shraeyas/DFL-Colab-Shraeyas
cd DFL-Colab-Shraeyas
!pip install -r requirements-colab.txt

Now, this will create a Folder Called **DFL-Colab-Shraeyas** on your **Google Drive**.

Inside that folder, there is a **workspace** folder.

You need to place your **data_dst.mp4** and **data_src.mp4** inside that folder.


------------------------------------------------------------------
#**Extract Frames From Video**
------------------------------------------------------------------


Extract Frames From **data_dst**

In [ ]:
!python main.py videoed extract-video --input-file workspace/data_dst.* --output-dir workspace/data_dst/

Extract Frames From **data_src**

In [ ]:
!python main.py videoed extract-video --input-file workspace/data_src.* --output-dir workspace/data_src/

------------------------------------------------------------------
#**Denoise**
------------------------------------------------------------------


Denoise Frames From **data_dst**

In [ ]:
!python main.py videoed denoise-image-sequence --input-dir workspace/data_dst

Denoise Frames From **data_src**

In [ ]:
!python main.py videoed denoise-image-sequence --input-dir workspace/data_src

------------------------------------------------------------------
#**Detect Faces and Align**
------------------------------------------------------------------


In [20]:
#@markdown Select the Type of **Detector** (s3fd recommended) and **FaceType**
Detector = "s3fd" #@param ["s3fd", "manual"]
FaceType = "whole_face" #@param ["head", "half_face", "full_face", "whole_face", "mark_only"]

In [21]:
#@markdown ###For **data_dst**
#@markdown <sup>**!python** main.py **extract** --input-dir workspace/data_dst --output-dir workspace/data_dst/aligned --face-type " + $FaceType + " --detector " + $Detector</sup>

cmd = "main.py extract --input-dir workspace/data_dst --output-dir workspace/data_dst/aligned --face-type " + FaceType + " --detector " + Detector
!python $cmd

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
python3: can't open file 'main.py': [Errno 2] No such file or directory


In [ ]:
#@markdown ###For **data_src**
#@markdown <sup>**!python** main.py **extract** --input-dir workspace/data_src --output-dir workspace/data_src/aligned --face-type " + $FaceType + " --detector " + $Detector</sup>

cmd = "main.py extract --input-dir workspace/data_src --output-dir workspace/data_src/aligned --face-type " + FaceType + " --detector " + Detector
!python $cmd

------------------------------------------------------------------
#**Sort Aligned Faces**
------------------------------------------------------------------


In [ ]:
#@markdown Select **sort type** (absdiff recommended)
sort_type = "absdiff" #@param ["absdiff","hist", "hist-dissim", "face-yaw", "face-pitch", "vggface", "blur", "final", "final-no-blur"]

In [ ]:
#@markdown For **data_dst**

#@markdown <sup>**!python** main.py **sort** --input-dir workspace/data_dst/aligned --by " + $sort_type

cmd = "main.py sort --input-dir workspace/data_dst/aligned --by "+sort_type
!python $cmd

In [ ]:
#@markdown For **data_src**

#@markdown <sup>**!python** main.py **sort** --input-dir workspace/data_src/aligned --by " + $sort_type

cmd = "main.py sort --input-dir workspace/data_src/aligned --by "+sort_type
!python $cmd

------------------------------------------------------------------
#**Faceset Enhance**
------------------------------------------------------------------

In [ ]:
#@markdown For **data_dst**

#@markdown <sup>**!python** main.py **facesettool** enhance --input-dir /workspace/data_dst/aligned</sup>
!python main.py facesettool enhance --input-dir /workspace/data_dst/aligned

In [ ]:
#@markdown For **data_src**

#@markdown <sup>**!python** main.py **facesettool** enhance --input-dir /workspace/data_src/aligned</sup>
!python main.py facesettool enhance --input-dir /workspace/data_src/aligned

------------------------------------------------------------------
#**Train Model**
------------------------------------------------------------------

In [ ]:
Model = "SAEHD" #@param ["SAEHD", "SAE", "Quick96", "H128", "LIAEF128", "DF", "AVATAR", "DEV_FANSEG"]

#@markdown <sup>**!python** main.py **train** **--training-data-src-dir** workspace/data_src/aligned **--training-data-dst-dir** workspace/data_dst/aligned **--pretraining-data-dir** pretrain **--model-dir** workspace/model **--model** " + $Model</sup>
cmd = "main.py train --training-data-src-dir workspace/data_src/aligned --training-data-dst-dir workspace/data_dst/aligned --pretraining-data-dir pretrain --model-dir workspace/model --model " + Model

!python $cmd

------------------------------------------------------------------
#**Convert frames**
------------------------------------------------------------------

In [ ]:
#@title Convert
Model = "SAEHD" #@param ["SAEHD", "SAE", "Quick96", "H128", "LIAEF128", "DF"]

#@markdown <sup>**!python** "main.py **convert** **--input-dir** workspace/data_dst/aligned **--output-dir** workspace/data_dst/merged **--model-dir** workspace/model **--model** " + $Model</sup>
if Model == "AVATAR":  
  cmd = "main.py convert --input-dir workspace/data_dst/aligned --output-dir workspace/data_dst/merged --model-dir workspace/model --model "+Model
else:
  cmd = "main.py convert --input-dir workspace/data_dst --output-dir workspace/data_dst/merged --aligned-dir workspace/data_dst/aligned --model-dir workspace/model --model "+Model

!python $cmd

------------------------------------------------------------------
#**Get Final Result Video**
------------------------------------------------------------------

In [ ]:
!python main.py videoed video-from-sequence --input-dir workspace/data_dst/merged --output-file workspace/result.mp4 --reference-file workspace/data_dst.mp4